In [17]:
import pandas as pd
import numpy as np

In [18]:
data = pd.read_csv("ELA_data_2013_2023.csv")

### **Data Feasibility Review & Data Cleaning/Preprocessing**

The initial data contains 626,462 rows and 18 columns. After filtering for years 2018, 2019, and 2022, the data contains 220,443 rows. Almost all columns do not have missing values, with the exception for school name. This is due to the data containing citywide and district level metrics, so school name is not applicable. 

The data was filtered to contain only years surrounding the COVID-19 pandemic, as this is the time period of interest. Unfortunately, data was not given for the years 2020 and 2021, due to cancelled and optional testing, respectively. All scores and levels (number and percentages) were converted to floats. The year was converted to an object. Groups with five or less tested students were suppressed with a value "s". Within this data, 58,872 rows are missing observations for scores and level breakdowns. These observations have been omitted.

In [19]:
from data_cleaning import cleaning_data, change_variable_type
school_data = cleaning_data(data) # filter for year, rename some vars

In [20]:
# school_data.info() # check

In [90]:
school_data.describe()

,number_tested,mean_scale_score,level_1_count,level_1_percentage,level_2_count,level_2_percentage,level_3_count,level_3_percentage,level_4_count,level_4_percentage,level_3_4_count,level_3_4_percentage
count,220443.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000,161571.000000
mean,248.897538,599.631240,77.352959,25.145654,96.179252,30.005807,92.342518,27.364279,68.441583,17.486974,160.784101,44.850499
std,3240.622795,11.607079,914.979254,20.025864,1107.334902,12.884432,1067.666357,14.803102,803.595139,17.560383,1849.885704,24.395932
min,1.000000,553.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,592.000000,3.000000,9.400000,6.000000,21.600000,5.000000,17.400000,1.000000,4.300000,7.000000,26.200000
50%,32.000000,599.000000,10.000000,21.000000,14.000000,30.300000,13.000000,26.500000,6.000000,12.500000,20.000000,43.300000
75%,81.000000,608.000000,25.000000,36.600000,32.000000,37.900000,31.000000,35.700000,20.000000,25.800000,52.000000,63.100000
max,396502.000000,650.000000,96610.000000,100.000000,118086.000000,100.000000,109742.000000,100.000000,77394.000000,100.000000,185973.000000,100.000000


In [21]:
# convert num columns
cols = ['mean_scale_score','level_1_count','level_2_count','level_3_count', 'level_4_count', 'level_4_percentage', 'level_3_4_count',
        'level_1_percentage','level_2_percentage','level_3_percentage','level_4_percentage','level_3_4_percentage']
school_data = change_variable_type(school_data,cols)
# change year to ob
school_data['Year'] = school_data['Year'].astype('object')

In [ ]:
school_data.info() # check new dtypes

<class 'pandas.core.frame.DataFrame'>
Index: 220443 entries, 7 to 626427
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Report Category         220443 non-null  object 
 1   Geographic Subdivision  220443 non-null  object 
 2   school_name             207431 non-null  object 
 3   Grade                   220443 non-null  object 
 4   Year                    220443 non-null  object 
 5   Student Category        220443 non-null  object 
 6   number_tested           220443 non-null  int64  
 7   mean_scale_score        161571 non-null  float64
 8   level_1_count           161571 non-null  float64
 9   level_1_percentage      161571 non-null  float64
 10  level_2_count           161571 non-null  float64
 11  level_2_percentage      161571 non-null  float64
 12  level_3_count           161571 non-null  float64
 13  level_3_percentage      161571 non-null  float64
 14  level_4_count           1

### **Ethics and Bias Considerations**

In [24]:
school_data['Student Category'].value_counts()

Student Category
All Students              15265
Not SWD                   15259
Econ Disadv               15251
Never ELL                 15239
Hispanic                  15204
Female                    15195
Male                      15172
SWD                       15064
Not Econ Disadv           14769
Black                     14434
Ever ELL                  14078
Current ELL               14015
White                     12758
Asian                     12609
Multi-Racial               8036
Native American            7843
Female Asian                 21
Female Black                 21
Female Hispanic              21
Female Multi-Racial          21
Male Black                   21
Male Asian                   21
Female White                 21
Female Native American       21
Male White                   21
Male Native American         21
Male Multi-Racial            21
Male Hispanic                21
Name: count, dtype: int64

In [25]:
school_data['Grade'].value_counts()

Grade
All Grades    52318
3             34353
4             34040
5             33785
6             22324
7             21919
8             21704
Name: count, dtype: int64

This data is very heavily balanced regarding different socioeconomic categories. There are some disparities with gender by ethnicity, however these indicators will not be used. The same can be said for grade, where there is sufficient distribution. 

### **Plots, Statistical Summaries, and Insights**

In [ ]:
import holoviews as hv
import hvplot.pandas

#### **Data Exploration for All Grades and All Students**

In [92]:
def plotting_sums(df):
    ''' Displays average mean scores per year. Returns a combined line and scatter'''
    filtered_df = df[df['Grade'] == 'All Grades']
    filtered_df = df[df['Report Category'] == 'School']
    filtered_df = filtered_df[filtered_df['Year'].isin([2018,2019,2022])]
    avg_score = filtered_df.groupby('Year')['mean_scale_score'].mean().reset_index()
    avg_score_line = avg_score.hvplot.line(x = 'Year', y = 'mean_scale_score', color = 'teal')
    avg_score_scatter = avg_score.hvplot.scatter(x = 'Year', y = 'mean_scale_score', color = 'teal')
    avg_score_combined = avg_score_scatter * avg_score_line
    avg_score_combined.opts(title = 'Average Mean Scale Score, per Year')
    return avg_score_combined


In [93]:
plotting_sums(school_data)

:Overlay
   .Scatter.I :Scatter   [Year]   (mean_scale_score)
   .Curve.I   :Curve   [Year]   (mean_scale_score)

This plot shows the mean score for ELA exams in 2018, 2019, and 2022. As shown above, the mean scores drop slightly from 2018 to 2019. From 2019 to 2022, the mean score increases by about one point.

In [29]:
# filter for years, school level, all grades

school_data_2019 = school_data[
    (school_data['Year'] == 2019) &
    (school_data['Report Category'] == 'School') &
    (school_data['Grade'] == 'All Grades')
]

school_data_2022 = school_data[
    (school_data['Year'] == 2022) &
    (school_data['Report Category'] == 'School') &
    (school_data['Grade'] == 'All Grades')
]

In [30]:
from holoviews import opts

In [31]:
# mean scale score distribution 2019 & 2022
p1 = school_data_2019.hvplot.hist(
    y = 'mean_scale_score',
    bins = 30,
    title = 'Mean Scale Score Distribution - 2019'
)

p2 = school_data_2022.hvplot.hist(

    y = 'mean_scale_score',
    bins = 30,
    title = 'Mean Scale Score Distribution - 2022'
)

plot = (p1 + p2).opts(opts.Histogram(xlabel = "Mean Scale Score"))
plot

:Layout
   .Histogram.I  :Histogram   [mean_scale_score]   (Count)
   .Histogram.II :Histogram   [mean_scale_score]   (Count)

In [32]:
print(school_data_2019['mean_scale_score'].mean().round(2))
print(school_data_2022['mean_scale_score'].mean().round(2))

598.88
599.8


The dataset has been filtered into two pieces of data, each containing relevant information for the years 2019 and 2022. With these datasets, we are looking at school level exam scores for all grades. For all students, the mean score increased by about one point. Nevertheless, this variable is normally distributed, making it easier to analyze. 

In [33]:
# levels over years
school_data.groupby('Year')[
    ['level_1_percentage',
     'level_2_percentage',
     'level_3_percentage',
     'level_4_percentage']
].mean().hvplot.line()

:NdOverlay   [Variable]
   :Curve   [Year]   (value)

This next plot shows percentages of the levels throughout the years. While students categorized in Level 4 are increasing, we can see subtle increases in Level 2 students from 2019 to 2022.

In [34]:
# does school size affect score
school_data_2019.hvplot.scatter(
    x = 'mean_scale_score',
    y = 'number_tested'
).opts(xlabel="Score", ylabel='Number Tested')

:Scatter   [mean_scale_score]   (number_tested)

This plot is meant to analyze if school size impacts scores. Schools with more students tested tend to have higher scoring students.

In [35]:
school_data_2019.hvplot.scatter(
    x  = 'mean_scale_score',
    y = 'level_3_4_percentage'
).opts(xlabel='Mean Score', ylabel='Level 3 and Level 4 %')

:Scatter   [mean_scale_score]   (level_3_4_percentage)

This plot shows the correlation between high performing students and mean score. Naturally, these variables are correlated due to the levels being reflected by the scores.

#### **Data Exploration by Grade**

In [74]:
school_data['Grade'].unique()

array(['3', '4', '5', '6', '7', '8', 'All Grades'], dtype=object)

notes:
- advantage disadvantage, how did they make up? 
- permanent damages? 

In [76]:
school_data_grade= school_data[
    (school_data['Year'].isin([2019, 2022])) &
    (school_data['Report Category'] == 'School') &
    (school_data['Grade'].isin(['3','4','5','6','7','8']))]

In [89]:
p_data3 = (school_data_grade.groupby(['Year','Grade'])[
    ['level_1_percentage',
     'level_2_percentage',
     'level_3_percentage',
     'level_4_percentage',
     'level_3_4_percentage',
     'mean_scale_score'
     ]
].mean())

p01 = p_data3.hvplot.line(
    x = 'Year',
    y = 'level_1_percentage',
    by = 'Grade'
).opts(ylabel = 'Level 1 %')

p02= p_data3.hvplot.line(
    x = 'Year',
    y = 'level_2_percentage',
    by = 'Grade'
).opts(ylabel = 'Level 2 %')

p03 = p_data3.hvplot.line(
    x = 'Year',
    y = 'level_3_percentage',
    by = 'Grade'
).opts(ylabel = 'Level 3 %')

p04= p_data3.hvplot.line(
    x = 'Year',
    y = 'level_4_percentage',
    by = 'Grade'
).opts(ylabel = 'Level 4 %')

p05 = p_data3.hvplot.line(
    x = 'Year',
    y = 'level_3_4_percentage',
    by = 'Grade'
).opts(ylabel = 'Level 3 & 4 %')

p06 = p_data3.hvplot.line(
    x = 'Year',
    y = 'mean_scale_score',
    by = 'Grade'
).opts(ylabel='Mean Score')

layout0 = (p01 + p02 + p03 + p04 + p05 + p06).cols(2)
layout0





:Layout
   .NdOverlay.I   :NdOverlay   [Grade]
      :Curve   [Year]   (level_1_percentage)
   .NdOverlay.II  :NdOverlay   [Grade]
      :Curve   [Year]   (level_2_percentage)
   .NdOverlay.III :NdOverlay   [Grade]
      :Curve   [Year]   (level_3_percentage)
   .NdOverlay.IV  :NdOverlay   [Grade]
      :Curve   [Year]   (level_4_percentage)
   .NdOverlay.V   :NdOverlay   [Grade]
      :Curve   [Year]   (level_3_4_percentage)
   .NdOverlay.VI  :NdOverlay   [Grade]
      :Curve   [Year]   (mean_scale_score)

- transitioning schools maybe?

These plots show level and score averages amongst all grades. Per the plots, particulary Plot 6, it can be seen that younger children in grades 3 and 4 suffered the most from learning disruptions post-COVID.

#### **Data Exploration for Socioeconomic Breakdowns**

This data contains a column called "Student Category". Below are the categories students are broken into, per the codebook provided by NYC OpenData. For the sake of this project, we will be looking at differences in gender, ethnicity, and economic status.

All Students <br>
SWD (Students with Disability): Not SWD, SWD <br>
<span style='color:#f69697'>**Ethnicity: Asian, Black, Hispanic, Multi-Racial, Native American, White** </span> <br>
<span style='color:#f69697'>**Gender: Female, Male, Neither Female nor Male** </span><br>
Gender by Ethnicity: Female or Male Asian, Female or Male Black, Female or Male Hispanic, Female or Male Multi-Racial, Female or Male Native American, Female or Male White <br>
<span style='color:#f69697'>**Econ Status (economic status): Econ Disadv, Not Econ Disadv**  </span> <br>
ELL (English Language Learner): Current ELL, Ever ELL, Never ELL" <br>


In [44]:
school_data_gender = school_data[
    (school_data['Year'].isin([2019, 2022])) &
    (school_data['Report Category'] == 'School') &
    (school_data['Grade'] == 'All Grades') &
    (school_data['Student Category'].isin(['Female','Male']))]

In [71]:
# levels over years
p_data = (school_data_gender.groupby(['Year','Student Category'])[
    ['level_1_percentage',
     'level_2_percentage',
     'level_3_percentage',
     'level_4_percentage',
     'level_3_4_percentage'
     ]
].mean())

p1 = p_data.hvplot.line(
    x = 'Year',
    y = 'level_1_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 1 %')

p2 = p_data.hvplot.line(
    x = 'Year',
    y = 'level_2_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 2 %')

p3 = p_data.hvplot.line(
    x = 'Year',
    y = 'level_3_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 3 %')

p4 = p_data.hvplot.line(
    x = 'Year',
    y = 'level_4_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 4 %')

p5 = p_data.hvplot.line(
    x = 'Year',
    y = 'level_3_4_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 3 & 4 %')

layout1 = (p1 + p2 + p3 + p4 + p5).cols(2)
layout1



:Layout
   .NdOverlay.I   :NdOverlay   [Student Category]
      :Curve   [Year]   (level_1_percentage)
   .NdOverlay.II  :NdOverlay   [Student Category]
      :Curve   [Year]   (level_2_percentage)
   .NdOverlay.III :NdOverlay   [Student Category]
      :Curve   [Year]   (level_3_percentage)
   .NdOverlay.IV  :NdOverlay   [Student Category]
      :Curve   [Year]   (level_4_percentage)
   .NdOverlay.V   :NdOverlay   [Student Category]
      :Curve   [Year]   (level_3_4_percentage)

The plots show a gendered breakdown of ELA levels from 2019 - 2022. Based on the fifth plot for combined Level 3 and Level 4 data, it can be implied that the increases in Level 2 may be due to the decreases in proficient levels. 

In [68]:
school_data_econ = school_data[
    (school_data['Year'].isin([2019, 2022])) &
    (school_data['Report Category'] == 'School') &
    (school_data['Grade'] == 'All Grades') &
    (school_data['Student Category'].isin(['Econ Disadv','Not Econ Disadv']))]

In [86]:
# levels over years
p_data2= (school_data_econ.groupby(['Year','Student Category'])[
    ['level_1_percentage',
     'level_2_percentage',
     'level_3_percentage',
     'level_4_percentage',
     'level_3_4_percentage',
     'mean_scale_score'
     ]
].mean())

p6 = p_data2.hvplot.line(
    x = 'Year',
    y = 'level_1_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 1 %')

p7 = p_data2.hvplot.line(
    x = 'Year',
    y = 'level_2_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 2 %')

p8 = p_data2.hvplot.line(
    x = 'Year',
    y = 'level_3_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 3 %')

p9 = p_data2.hvplot.line(
    x = 'Year',
    y = 'level_4_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 4 %')

p10 = p_data2.hvplot.line(
    x = 'Year',
    y = 'level_3_4_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 3 & 4 %')

p11 = p_data2.hvplot.line(
    x = 'Year',
    y = 'mean_scale_score',
    by = 'Student Category'
).opts(ylabel = 'Mean Score')

layout2 = (p6 + p7 + p8 + p9 + p10 + p11).cols(2)
layout2

:Layout
   .NdOverlay.I   :NdOverlay   [Student Category]
      :Curve   [Year]   (level_1_percentage)
   .NdOverlay.II  :NdOverlay   [Student Category]
      :Curve   [Year]   (level_2_percentage)
   .NdOverlay.III :NdOverlay   [Student Category]
      :Curve   [Year]   (level_3_percentage)
   .NdOverlay.IV  :NdOverlay   [Student Category]
      :Curve   [Year]   (level_4_percentage)
   .NdOverlay.V   :NdOverlay   [Student Category]
      :Curve   [Year]   (level_3_4_percentage)
   .NdOverlay.VI  :NdOverlay   [Student Category]
      :Curve   [Year]   (mean_scale_score)

These plots show the level breakdowns between economically and not economically disadvantaged students. Trending with my assumptions, economically disadvantaged students have a decrease in ELA proficiency. Generally, mean scores increased across all students. 

In [73]:
school_data['Student Category'].unique()

array(['All Students', 'Ever ELL', 'Econ Disadv', 'Not Econ Disadv',
       'Never ELL', 'Current ELL', 'Asian', 'Black', 'Hispanic',
       'Multi-Racial', 'Native American', 'White', 'Female Asian',
       'Female Black', 'Female Hispanic', 'Female Multi-Racial',
       'Female Native American', 'Female White', 'Male Asian',
       'Male Black', 'Male Hispanic', 'Male Multi-Racial',
       'Male Native American', 'Male White', 'Female', 'Male', 'Not SWD',
       'SWD'], dtype=object)

In [87]:
school_data_race = school_data[
    (school_data['Year'].isin([2019, 2022])) &
    (school_data['Report Category'] == 'School') &
    (school_data['Grade'] == 'All Grades') &
    (school_data['Student Category'].isin(['Asian','Black','Hispanic','Multi-Racial','Native American','White']))]

In [88]:
# levels over years
p_data4= (school_data_race.groupby(['Year','Student Category'])[
    ['level_1_percentage',
     'level_2_percentage',
     'level_3_percentage',
     'level_4_percentage',
     'level_3_4_percentage',
     'mean_scale_score'
     ]
].mean())

p12= p_data4.hvplot.line(
    x = 'Year',
    y = 'level_1_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 1 %')

p13 = p_data4.hvplot.line(
    x = 'Year',
    y = 'level_2_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 2 %')

p14 = p_data4.hvplot.line(
    x = 'Year',
    y = 'level_3_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 3 %')

p15 = p_data4.hvplot.line(
    x = 'Year',
    y = 'level_4_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 4 %')

p16 = p_data4.hvplot.line(
    x = 'Year',
    y = 'level_3_4_percentage',
    by = 'Student Category'
).opts(ylabel = 'Level 3 & 4 %')

p17 = p_data4.hvplot.line(
    x = 'Year',
    y = 'mean_scale_score',
    by = 'Student Category'
).opts(ylabel = 'Mean Score')

layout2 = (p12 + p13 + p14 + p15 + p16 + p17).cols(2)
layout2

:Layout
   .NdOverlay.I   :NdOverlay   [Student Category]
      :Curve   [Year]   (level_1_percentage)
   .NdOverlay.II  :NdOverlay   [Student Category]
      :Curve   [Year]   (level_2_percentage)
   .NdOverlay.III :NdOverlay   [Student Category]
      :Curve   [Year]   (level_3_percentage)
   .NdOverlay.IV  :NdOverlay   [Student Category]
      :Curve   [Year]   (level_4_percentage)
   .NdOverlay.V   :NdOverlay   [Student Category]
      :Curve   [Year]   (level_3_4_percentage)
   .NdOverlay.VI  :NdOverlay   [Student Category]
      :Curve   [Year]   (mean_scale_score)

### **Data Limitations**

As discussed previously, there are some missing values in this dataset. Smaller schools have fewer testing students, which leads to that information being suppressed by the authors of the data. This is unfortunate, as the EDA reveals that smaller schools suffer the most from low testing scores. Another limitation is the missing data from 2020 and 2021 due to in-person learning restrictions during the pandemic. Since I am looking at 2022 as the post period, there may be some bounceback from students who fell behind during the pandemic. However, this is not to say that residual effects do not bleed into post-pandemic years. 

- look at grade level suppressed values and how to combat

### **Implications for Methods**

The two-way fixed effects difference-in-difference model relies on the parallel trends assumption. In other words, treatment and nontreatment need to have constant, parallel trends in order to satisfy the model conditions. While I have not decided on a way to define my treatment (citywide median, district median, 25% quantile, for scores and/or levels etc.), I am confident in the model and may have to include more fixed effects or interaction terms to combat a potential issue. 